### Importowanie pakietów



In [10]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Wyslanie żadania dostepu do strony internetowej z opiniami o produkcie 

In [ ]:
with open ("/headers.json","r") as jf:
    headers = json.load(jf)


In [ ]:
product_id = "84514582"
url = "https://www.ceneo.pl/84514582#tab=reviews"
all_opinions = []
response = requests.get(url,headers=headers)
response.status_code

### Jeżeli operacja zakończy sie powodzeniem

In [13]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

### Dla każdej opinii wyodrębnienie z kodu HTML poszzczególnych skladowych i przypisanie ich do elementów złożonej struktury danych

In [ ]:
for opinion in opinions:
    try:
        single_opinion = {
            "author": opinion.select_one("span.user-post__author-name").text.strip(),
            "recomendation":opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
            "stars":opinion.select_one("span.user-post__score-count").text.strip(),
            "content":opinion.select_one("div.user-post__text").text.strip(),
            "pros":[p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")],
            "cons":[c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")],
            "useful":opinion.select_one("button.vote-yes > span").text.strip(),
            "useless":opinion.select_one("button.vote-no > span").text.strip(),
            "post_date":opinion.select_one("span.user-post__published > time:nth-child(1)[datetime]").text.strip(),
            "purchase_date":opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]").text.strip(),
        }
        all_opinions.append(single_opinion)

    except (AttributeError, TypeError):
        pass

In [ ]:
print(type(single_opinion["author"]))


### Jeśli istnieje kolejna strona z opiniami, przejscie do niej i powtórzenie dla niej kroków 1-4 


In [ ]:
product_id = "84514582"
next_page = "https://www.ceneo.pl/84514582#tab=reviews"
all_opinions = []
while next_page:
    response = requests.get(next_page, headers=headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
             try:
                single_opinion = {
                    "author": opinion.select_one("span.user-post__author-name").text.strip(),
                    "recomendation":opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
                    "stars":opinion.select_one("span.user-post__score-count").text.strip(),
                    "content":opinion.select_one("div.user-post__text").text.strip(),
                    "pros":[p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")],
                    "cons":[c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")],
                    "useful":opinion.select_one("button.vote-yes > span").text.strip(),
                    "useless":opinion.select_one("button.vote-no > span").text.strip(),
                    "post_date":opinion.select_one("span.user-post__published > time:nth-child(1)[datetime]").strip(),
                    "purchase_date":opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]").strip(),
                }
                all_opinions.append(single_opinion)
             except (AttributeError, TypeError):
                pass
             try:
                next_page = "https://www.ceneo.pl/"
                page_dom.select_one("a.pagination__next")["href"]
             except TypeError:
                next_page = None
else: print (response.status_code)


### Zapisywanie wyników do bazy danych 

In [ ]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [ ]:
with open(f'./opinions{product_id}.json',"W",encoding="utf-8") as jf:
     json.dump(all_opinions, jf, ensure_ascii=False, indent=4)